# Importar Librerías

In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
import copy 

# Abrir archivos (csv, txt o xslx)

In [ ]:
def abrir_archivo():
    root = tk.Tk()
    root.withdraw()  # Ocultar la ventana principal

    tipos_archivos = [
        ("Archivos CSV", "*.csv"),
        ("Archivos de texto", "*.txt"),
        ("Archivos Excel", "*.xlsx")
    ]
    
    ruta_archivo = filedialog.askopenfilename(filetypes=tipos_archivos)
    
    if ruta_archivo:
        if ruta_archivo.endswith('.csv'):
            data = pd.read_csv(ruta_archivo)
        elif ruta_archivo.endswith('.txt'):
            data = pd.read_csv(ruta_archivo, delimiter='\t')  # Puedes cambiar el delimitador si es necesario
        elif ruta_archivo.endswith('.xlsx'):
            data = pd.read_excel(ruta_archivo)
        else:
            print("Formato de archivo no válido")
            return None, None, None
        
        return data, ruta_archivo  # Devolver el DataFrame y la ruta del archivo
    else:
        print("Ningún archivo seleccionado")
    
    root.destroy()

# Análisis del archivo

In [ ]:
#data = data.drop('DEPTH', axis=1)

In [ ]:
# Llamar a la función para cargar el archivo y obtener el DataFrame y la ruta del archivo
data, ruta_archivo = abrir_archivo()

In [ ]:
data.describe().apply(lambda s: s.apply('{0:.1f}'.format))

# Análisis de valores nulos

In [ ]:
def contar_nans(data, valor_a_nan=None):
    if valor_a_nan is not None:
        # Reemplazar el valor especificado por NaN
        data_con_nans = data.replace(valor_a_nan, np.nan)
        cantidad_nans = data_con_nans.isnull().sum()
    else:
        # Contar los valores específicos como NaN y luego contar los NaN por columna
        nan_values = [None, '', 'N/A', 'No Info']  # Valores que denotan ausencia de información
        cantidad_nans = data.isin(nan_values).sum()

    # Crear una tabla con la cantidad de NaN por columna
    tabla_nans = pd.DataFrame({'Cantidad de NaN': cantidad_nans})
    
    # Calcular el porcentaje de NaN por columna
    total_datos = len(data)
    tabla_nans['% de NaN'] = (tabla_nans['Cantidad de NaN'] / total_datos) * 100
    
    # Asignar categorías según el porcentaje de NaN
    tabla_nans['Score'] = pd.cut(tabla_nans['% de NaN'], bins=[-1, 25, 50, 75, 100],
                                 labels=['Muy bajo', 'Bajo', 'Moderado', 'Alto'])
    
    # Redondear los valores a 2 decimales
    tabla_nans = tabla_nans.round(2)
    
    return tabla_nans, data_con_nans # Retornar la tabla sin imprimir directamente

In [ ]:
# Establecer el valor que deseas tratar como NaN
valor_para_nan  = -999.25

In [ ]:
# Llamar a la función contar_nans() con el valor_para_nan especificado
tabla_nans, data_con_nans = contar_nans(data, valor_para_nan)

## Cantidad de valores nulos

In [ ]:
tabla_nans

## Archivo sin valores nulos

In [ ]:
data_2 = data_con_nans.dropna()
data_2.describe().apply(lambda s: s.apply('{0:.1f}'.format))

In [ ]:
# Obtener estadísticas descriptivas para df1 y df2
desc_df1 = data.describe().apply(lambda s: s.apply('{0:.1f}'.format))
desc_df2 = data_2.describe().apply(lambda s: s.apply('{0:.1f}'.format))

# Comparar los resultados de describe()
diferencias = desc_df1.compare(desc_df2, result_names=('Original', 'Sin Nulos'))
diferencias.T

# Análisis estadístico de los datos (a profundidad)

In [ ]:
def calcular_estadisticas(data_2):
    # Calcular estadísticas para cada columna del DataFrame
    estadisticas = {
        'Número de muestras': data_2.count(),
        'Valor mínimo': data_2.min(),
        'Valor máximo': data_2.max(),
        'Media': data_2.mean(),
        'Mediana': data_2.median(),
        'Primer cuartil': data_2.quantile(0.25),
        'Tercer cuartil': data_2.quantile(0.75),
        'Rango': data_2.max() - data_2.min(),
        'Rango intercuartil': data_2.quantile(0.75) - data.quantile(0.25),
        'Varianza': data_2.var(),
        'Desviación estándar': data_2.std(),
        'Coeficiente de variación': data_2.std() / data.mean(),
        'Simetría': data_2.skew(),
        'Curtosis': data_2.kurtosis()
    }
    
    estadisticas_df = pd.DataFrame(estadisticas)
    # Redondear los valores a 4 decimales
    estadisticas_df = estadisticas_df.round(4)
    return estadisticas_df

In [ ]:
# Utilizar la función con tu DataFrame 'data'
resultados_estadisticas = calcular_estadisticas(data_2)
resultados_estadisticas.T

## Análisis gráfico de los datos (histogramas y boxplots)

In [ ]:
def plot_hist_box(df):
    for col in df.columns:
        # Crear un subplot con 2 filas y 1 columna
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8, 6), gridspec_kw={'height_ratios': [1, 3]})

        # Generar un histograma con la media y la mediana marcadas
        df[col].plot.hist(ax=ax2, bins=10, alpha=0.5)
        ax2.axvline(df[col].mean(), color='red', linestyle='dashed', linewidth=1)
        ax2.axvline(df[col].median(), color='green', linestyle='dashed', linewidth=1)

        # Generar un gráfico de boxplot en horizontal arriba del histograma
        df[col].plot.box(ax=ax1, vert=False, patch_artist=True, boxprops=dict(facecolor='blue', color='black'), 
                         medianprops=dict(color='white'), whiskerprops=dict(color='black'), capprops=dict(color='black'))

        # Configurar los ejes y la leyenda
        ax2.set_xlabel(col)
        ax2.set_ylabel('Frecuencia')
        ax2.legend([col,'Media', 'Mediana'])
        ax1.legend([col])
        ax1.set_yticklabels('')
        

        # Compartir el eje X
        ax1.get_shared_x_axes()

        plt.show()

In [ ]:
plot_hist_box(data_2)

# Análisis de valores atípicos

## Método del Rango Intercuartílico (IQR):

In [ ]:
def remove_outliers_iqr(df):
    clean_data = df.copy()  # Copia del DataFrame para no modificar el original
    
    for col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR
        
        clean_data[col] = df[col][(df[col] >= lower_limit) & (df[col] <= upper_limit)]
    
    return clean_data

In [ ]:
data_2_iqr = data_2.copy()

In [ ]:
data_2_nol_iqr = remove_outliers_iqr(data_2_iqr)

## Estadígrafos sin valores atípicos (Método IQR)

In [ ]:
resultados_estadisticas_nol_iqr = calcular_estadisticas(data_2_nol_iqr)
resultados_estadisticas_nol_iqr.T

## Análisis gráfico de los datos sin valores atípicos (Método IQR; histogramas y boxplots)

In [ ]:
plot_hist_box(data_2_nol_iqr)

## Método basado en la Desviación Estándar (DE)

In [ ]:
def remove_outliers_std(df, n=2):
    clean_data = df.copy()  # Copia del DataFrame para no modificar el original
    
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        
        lower_limit = mean - n * std
        upper_limit = mean + n * std
        
        clean_data[col] = df[col][(df[col] >= lower_limit) & (df[col] <= upper_limit)]
    
    return clean_data

In [ ]:
data_2_std = data_2.copy()

In [ ]:
data_2_nol_std = remove_outliers_std(data_2_std)

## Estadígrafos sin valores atípicos (Método DE)

In [ ]:
resultados_estadisticas_nol_std = calcular_estadisticas(data_2_nol_std)
resultados_estadisticas_nol_std.T

## Análisis gráfico de los datos sin valores atípicos (Método IQR; histogramas y boxplots)

In [ ]:
plot_hist_box(data_2_nol_std)

# Análisis Estadístico Bivariado

In [ ]:
data_3 = data_2.copy()

In [ ]:
def corr_func(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top right hand corner of a plot.
    """
    r, _ = pearsonr(x, y)
    s, _ = spearmanr(x, y)
    k, _ = kendalltau(x, y)
    fontsize = 30
    ax = ax or plt.gca()
    ax.annotate(f'P = {r:.2f}', xy=(.5, .70), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')
    ax.annotate(f'S = {s:.2f}', xy=(.5, .50), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')
    ax.annotate(f'K = {k:.2f}', xy=(.5, .30), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')

## Valores de correlación de las variables y gráfico de Dispersión 

In [ ]:
g = sns.PairGrid(data_3)
g.map_upper(corr_func)
g.map_lower(sns.scatterplot)
g.map_diag(sns.histplot)
g.fig.suptitle("Gráfico de correlación", fontsize=50, y=1.05)
for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=29)
    ax.set_ylabel(ax.get_ylabel(), fontsize=29)
plt.show()

# Analisis Multivariado

In [ ]:
data_4 = data_2

In [ ]:
scaler = StandardScaler()
datascaled = scaler.fit_transform(data_4)
data_scaled = pd.DataFrame(datascaled)
data_scaled.columns = data_4.columns

In [ ]:
data_scaled.describe().apply(lambda s: s.apply('{0:.4f}'.format)).T

## Análisis multivariado de valores atípicos

In [ ]:
#Robust Mahalonibis Distance
def robust_mahalanobis_method(df):
    #Minimum covariance determinant
    rng = np.random.RandomState(0)
    real_cov = np.cov(df.values.T)
    X = rng.multivariate_normal(mean=np.mean(df, axis=0), cov=real_cov, size=506)
    cov = MinCovDet(random_state=0).fit(X)
    mcd = cov.covariance_ #robust covariance metric
    robust_mean = cov.location_  #robust mean
    inv_covmat = sp.linalg.inv(mcd) #inverse covariance metric
    
    #Robust M-Distance
    x_minus_mu = df - robust_mean
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    md = np.sqrt(mahal.diagonal())
    
    #Flag as outlier
    outlier = []
    C = np.sqrt(chi2.ppf((1-0.001), df=df.shape[1]))#degrees of freedom = number of variables
    for index, value in enumerate(md):
        if value > C:
            outlier.append(index)
        else:
            continue
    return outlier, md

In [ ]:
outliers_mahal_rob_bi, md_rb_bi = robust_mahalanobis_method(df=data_scaled)

In [ ]:
data_scaled_df = copy.deepcopy(data_scaled) 

In [ ]:
def flag_outliers(df, outliers):
    flag = []
    for index in range(df.shape[0]):
        if index in outliers:
            flag.append(-1)
        else:
            flag.append(1)
    return flag

In [ ]:
y_pred = flag_outliers(data_scaled_df, outliers_mahal_rob_bi)

In [ ]:
data_scaled['outliers'] = flag_outliers(data_scaled_df, outliers_mahal_rob_bi)

In [ ]:
y_pred = np.array(y_pred)

## Cantidad de Valores atípicos

In [ ]:
n_inliers = (y_pred == 1).sum()
n_outliers = (y_pred == -1).sum()
n_total = n_inliers + n_outliers

print(f"Number of inliers: {n_inliers}")
print(f"Number of outliers: {n_outliers}")
print(f"Total: {n_total}")

In [ ]:
labels = 'Inliers', 'Outliers'
sizes = [n_inliers, n_outliers]
plt.rcParams["figure.facecolor"] = "w"
plt.figure(figsize=(6,6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('Distribución de datos y atipicos con \nDistancia de Mahalanobis',size=16)
plt.show()

In [ ]:
ds = data_scaled_df.copy()
ds.columns=data_scaled_df.columns
ds['legend'] = y_pred

## Diagrama de dispersión de datos sin valores atípicos

In [ ]:
# Crear un gráfico de Seaborn
plt.rc('legend', fontsize=15, title_fontsize=17)
plt.rc('axes', titlesize=25)
sns.pairplot(ds, hue='legend', palette='bright', corner=True)
plt.suptitle('Bivariate Plot of Variables with Outliers', size=24, y=1.05)
plt.show()